In [12]:
import csv
import random
import math

In [13]:
# creating the function to load the dataset.
def LoadDataset(filename):
    dataset = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            dataset.append(row)
    return dataset

In [38]:
def reviews_load(filename):
    reviews = []
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            reviews.append(row)
    return reviews

In [14]:

# then split the dataset into train, dev, and test sets
def split_train_dev_eval(dataset, split_ratio):
    train_size = int(len(dataset) * split_ratio) # split the train according to input ratio
    train_dataset = []
    devandeval = dataset[:]
    while len(train_dataset) < train_size:
        index = random.randrange(len(devandeval))
        train_dataset.append(devandeval.pop(index))
    return [train_dataset, devandeval, devandeval] #return list contains the three parts.

In [15]:
# function that calculates the word occurence in all reviews
def word_occurrences(dataset, vocabulary):
    WordOccurrences = {}
    for word in vocabulary:
        WordOccurrences[word] = 0
        for document in dataset:
            if word in document[0].split():
                WordOccurrences[word] += 1
    return WordOccurrences

In [16]:

# function that calculates the probability of each class (fresh or rotten)
def ClassProbabilities(dataset):

    class_probabilities = {}
    
    for document in dataset:
        sentiment = document[0]
        if sentiment == "Freshness": # ignoring the excess lables from the dataset.
            pass
        else:

            if sentiment not in class_probabilities:
                class_probabilities[sentiment] = 1 
            else:
                class_probabilities[sentiment] += 1 #counting the number of reviews by adding ones in each loop trun.
    total_documents = len(dataset) #the number of all reviews
    
    for sentiment in class_probabilities:
        class_probabilities[sentiment] /= total_documents
        
        
    return class_probabilities #returns dictionary of both classes probabilities.

In [17]:
# calculating the conditional probability of each word given the class.
def word_conditional_probability(dataset, vocabulary, word_occurrences):
    conditional_probabilities = {}
    total_words_per_class = {}
    for sentiment in ["fresh","rotten"]:
        total_words_per_class[sentiment] = 0
        for word in vocabulary:
            total_words_per_class[sentiment] += word_occurrences[word][sentiment] + 1
    for word in vocabulary:
        conditional_probabilities[word] = {}
        for sentiment in ["fresh","rotten"]:
            conditional_probabilities[word][sentiment] = (word_occurrences[word][sentiment] + 1) / total_words_per_class[sentiment]
    return conditional_probabilities #returns dictionary of each word and its corrseponding probability in both classes.

In [18]:
# here we make predictions according to the model we created.
def predict(document, vocabulary, class_probabilities, conditional_probabilities):
    words = document.split()
    positive_probability = math.log(class_probabilities['fresh'])
    negative_probability = math.log(class_probabilities['rotten'])
    for word in words:
        if word in vocabulary:
            positive_probability += math.log(conditional_probabilities[word]['fresh'])
            negative_probability += math.log(conditional_probabilities[word]['rotten'])
    if positive_probability > negative_probability:
        return 'fresh'
    else:
        return 'rotten'
   

In [29]:
# then lets evaluate our model.
def evaluate(dataset, vocabulary, class_probabilities, conditional_probabilities):
    correct_predictions = 0
    for document in dataset:
        predicted_sentiment = predict(document[1], vocabulary, class_probabilities, conditional_probabilities)
        if predicted_sentiment == document[0]:
            correct_predictions += 1
    accuracy = correct_predictions / len(dataset)
    return accuracy

In [39]:
# the main function
def main():
    
    #loading the dataset
    dataset = LoadDataset('rt_reviews.csv')

    # Split the dataset into train, dev, and test sets
    train_set, dev_set, test_set = split_train_dev_eval(dataset, 0.6)
    
    vocab_list =['the', 'a', 'of', 'and', 'to', 'is', 'in', 'that', 'it', 'The', 'as', 'with', 'but', 'for', 'its', 'this', 'film', 'an', 'on', 'be', 'A', 'movie', 'not', 'you', 'by', "it's", 'are', 'at', 'more', 'from', 'than', 'about', 'has', 'like', 'his', 'have', "It's", 'one', 'all', 'so', 'or', 'I', 'just', 'into', 'much', 'This', 'up', 'most', 'It', 'story', '-', 'who', 'out', 'will', 'too', 'what', 'some', 'even', 'if', '--', 'can', 'their', 'make', 'no', 'which', 'only', 'good', 'was', "doesn't", 'never', 'little', 'been', 'when', 'he', 'very', 'any', 'they', 'best', 'makes', 'enough', '...', 'her', 'may', 'how', 'we', 'If', 'would', 'comedy', 'But', 'your', 'way', 'could', "that's", 'time', "isn't", 'us', 'feels', 'action', 'characters', 'does', '[Full', 'get', 'many', 'something', 'also', 'great', 'Spanish]', 'might', 'really', 'first', 'still', 'it.', 'An', 'two', 'through', 'being', 'In', 'see', 'made', 'there', 'should', 'those', 'do', 'better', 'between', 'director', 'review', 'own', 'without', 'over', 'new', 'As', 'while', 'every', 'well', 'work', 'feel', 'both', 'film,', 'off', "can't", 'such', 'drama', 'films', 'seems', 'quite', 'movies', 'character', "don't", 'other', 'love', 'less', 'performance', "film's", 'real', 'look', 'film.', 'plot', 'bad', 'fun', 'few', 'horror', 'life', 'had', 'sense', "there's", 'because', 'kind', 'movie.', 'nothing', 'often', 'almost', 'people', "There's", 'There', 'take', 'funny', 'yet', 'lot', 'them', 'ever', 'another', 'far', 'long', 'these', 'where', 'performances', 'While', 'hard', 'rather', 'What', 'comes', 'And', 'For', 'cast', 'our', 'thriller', 'takes', 'down', 'find', 'movie,', 'bit', 'script', 'same', 'gets', 'You', 'big', 'emotional', 'come', 'going', 'tale', 'thing', 'entertaining', 'here', 'go', 'know', 'right', "you're", 'after', 'were', 'world', 'least', 'moments', 'seen', 'back', 'With', 'anything', 'audience', 'worth', 'then', 'want', 'American', 'making', 'documentary', 'things', 'original', 'think', 'pretty', 'though', 'family', 'ultimately', 'end', 'romantic', 'human', 'One', 'narrative', 'interesting', 'me', 'say', 'cinematic', '"The', 'before', 'watching', 'around', 'gives', 'piece', 'scenes', 'full', 'always', 'keep', 'set', 'itself', 'old', 'Not', 'Hollywood', 'Even', 'genre', 'young', 'she', 'seem', 'my', 'visual', 'actors', 'whole', 'need', 'actually', 'comic', 'Despite', 'goes', 'Review', 'it,', 'give', 'last', 'fans', 'turns', 'man', 'simply', 'is,', 'becomes', 'once', 'probably', 'trying', 'strong', 'true', 'part', 'heart', 'plays', "he's", 'time.', 'everything', 'watch', 'screen', 'Like', 'series', 'minutes', 'picture', 'truly', 'point', 'humor', 'story.', 'At', 'high', 'whose', 'him', 'along', 'since', 'sometimes', 'left', 'away', 'feature', 'certainly', 'lacks', 'times', 'works', 'story,', 'compelling', 'together', 'each', 'why', "you'll", 'mostly', 'final', 'offers', 'feeling', 'looks', 'dialogue', "I'm", "movie's", 'especially', 'fails', 'portrait', 'manages', 'sort', 'delivers', 'years', 'acting', 'lack', 'version', 'material', "won't", 'When', 'special', 'classic', 'sequel', 'style', 'and,', 'idea', 'worst', 'dark', 'one.', 'political', 'despite', 'funny,', 'John', 'anyone', 'fine', 'half', 'play', 'All', 'de', 'three', 'become', 'familiar', 'power', 'keeps', 'turn', 'wants', 'experience', 'got', 'effects', 'that,', 'easy', 'filmmakers', 'leave', 'put', 'short', 'them.', 'laughs', "didn't", 'beyond', 'fascinating', 'instead', 'nearly', 'ends', 'looking', 'exactly', 'star', 'place', 'beautiful', 'completely', 'result', 'show', 'animated', 'Director', 'title', 'music', 'modern', 'shows', 'second', 'comedy.', 'subject', 'Though', 'done', 'solid', 'Michael', 'dramatic', 'powerful', 'proves', 'leaves', 'sure', 'tries', 'under', 'likely', 'particularly', 'We', 'moving', 'social', "they're", 'enjoyable', 'perfect', 'given', 'way.', 'against', 'kids', 'history', 'enjoy', 'viewers', 'audiences', 'found', 'remains', '2', 'engaging', 'plenty', 'year.', 'let', 'lost', 'nor', 'behind', 'No', 'different', 'already', 'until', 'predictable', 'tone', 'past', 'now', '&', 'characters,', 'premise', 'did', 'franchise', 'scene', 'comedy,', 'be.', 'adaptation', 'art', 'By', 'help', 'latest', 'sci-fi', 'must', 'ideas', 'end,', 'fun.', 'production', 'cinema', 'lead', 'entirely', 'direction', 'adventure', 'So', 'attempt', 'jokes', 'elements', 'having', 'filmmaker', 'getting', 'clever', 'here,', 'problem', 'surprisingly', 'running', 'effort', 'shot', 'perhaps', 'That', 'life.', 'Mr.', 'here.', 'someone', 'deeply', 'To', "you've", 'rare', 'occasionally', 'tell', 'time,', 'directed', 'romance', 'summer', 'reason', 'expect', 'fantasy', 'fact', 'needs', 'use', 'book', 'war', 'New', 'impressive', 'role', 'next', 'women', 'is.', 'silly', 'recent', 'entertainment', 'Just', 'personal', 'care', 'matter', 'satisfying', 'close', 'easily', "I've", 'emotionally', 'serious', 'flick', "aren't", 'falls', 'small', 'Although', 'dull', 'work.', 'epic', 'debut', 'light', 'beautifully', 'playing', 'smart', 'doing', 'start', 'musical', 'characters.', 'Man', 'during', 'again', 'black', 'everyone', 'Its', 'James', 'moment', 'TV', 'important', 'filmmaking', 'fresh', 'case', 'level', 'screenplay', 'mind', 'message', 'After', 'lives', 'line', 'intriguing', 'approach', 'brings', 'finally', 'movies.', 'whether', 'thought', 'act', 'terrific', 'charm', 'difficult', 'storytelling', 'central', 'across', 'drama.', "we're", 'actor', 'crime', 'complex', '.', 'violence', 'animation', 'stories', 'funny.', 'effective', 'seeing', 'David', 'all.', 'else', 'entire', 'written', 'couple', 'thanks', 'exercise', 'obvious', 'finds', 'early', 'clearly', 'hope', 'fully', 'woman', 'sweet', 'third', 'either', 'Is', 'taking', 'live', 'wonder', 'study', 'within', 'ways', 'energy', 'deserves', 'exciting', 'rest', 'above', 'superhero', 'ending', 'neither', 'himself', 'men', 'sequences', 'screen.', 'hours', 'wrong', 'genuine', 'game', 'talent', 'bring', 'hit', 'simple', 'barely', 'fun,', 'job', 'home', 'camera', 'visually', 'based', 'certain', 'nice', 'utterly', 'face', 'single', 'among', '(and', 'moral', 'wish', 'decent', 'perfectly', 'remake', 'Too', 'knows', '...a', 'call', 'starts', 'period', 'Unfortunately,', 'nature', 'year', 'charming', 'deep', 'clear', 'How', 'tension', 'More', 'attention', 'definitely', 'filled', "year's", 'brilliant', 'believe', 'satire', 'out.', 'rarely', 'able', 'talented', "what's", 'main', 'appeal', 'teen', 'all,', 'somewhat', 'stars', 'told', 'female', 'films,', 'He', 'drama,', 'means', 'thriller.', 'British', 'novel', 'coming', 'cast,', 'played', 'way,', 'worthy', 'moments,', 'taken', 'sex', 'laugh', 'sound', 'slightly', 'amusing', 'movies,', 'creative', 'unique', 'previous', 'excellent', 'felt', 'greatest', 'genuinely', 'quality', 'merely', 'viewer', 'work,', 'mess', 'used', 'create', 'although', 'highly', 'Some', 'largely', 'thoroughly', 'rich', 'psychological', 'My', 'wonderful', 'toward', 'good,', 'suspense', 'films.', 'They', 'run', 'again.', '[A]', 'War', 'parts', 'form', 'eyes', 'potential', 'top', 'Disney', 'performances,', 'giving', 'spirit', 'provides', 'slow', 'times,', 'remarkable', 'good.', 'Oscar', 'deliver', 'stuff', 'maybe', 'absolutely', 'contemporary', 'focus', 'mix', 'Robert', 'try', 'On', 'interest', 'you.', 'chemistry', 'relationship', 'Paul', 'several', 'French', 'magic', "couldn't", "you'd", 'melodrama', 'features', 'that.', 'themes', 'experience.', 'mystery', 'side', 'hour', 'video', 'hits', 'example', 'head', 'historical', 'boring', 'well,', 'stunning', 'usual', 'impossible', 'pop', 'hardly', 'hold', 'up.', 'save', 'leads', 'itself.', 'major', 'life,', 'this.', 'complete', 'offer', 'Of', 'amount', 'inspired', 'memorable', 'end.', 'actual', 'scary', 'school', 'happens', 'well.', 'pure', 'questions', 'journey', 'Night', 'lacking', 'fall', 'working', 'fairly', 'age', 'eye', 'children', 'action,', 'better.', 'thriller,', 'career', 'hilarious', 'themselves', 'Tom', 'disappointing', 'captures', 'reality', 'supposed', 'genre.', 'none', 'minutes,', 'performances.', 'more.', 'watch.', 'que', 'concept', 'however,', 'ride', 'quickly', 'telling', 'audience.', 'created', 'add', 'Star', 'loses', 'Love', "we've", 'cast.', 'cheap', 'performance.', 'inside', 'depth', 'sets', 'points', 'Will', 'pace', "wouldn't", 'gone', 'attempts', 'mean', 'heart.', 'said', 'Black', 'beauty', 'violent', 'writing', 'vision', 'biggest', 'years.', 'overly', "wasn't", 'bland', 'be,', 'culture', 'generic', 'biopic', 'wit', 'overall', 'surprising', 'otherwise', 'terrible', 'dumb', 'succeeds', 'one,', "I'd", "Don't", 'except', 'world.', 'too.', 'ambitious', 'though,', 'surprise', 'stand', 'script,', 'enough.', 'images', 'question', 'flat', 'guy', 'intelligent', 'twists', 'somehow', 'From', 'possible', 'sit', "'The", 'death', 'cinema.', 'needed', 'low', "That's", 'extremely', 'strange', 'issues', 'upon', 'fan', 'Christmas', 'Last', 'suffers', 'crafted', 'called', 'gags', 'visuals', 'lines', 'yourself', 'Day', 'conventional', 'uses', 'hero', 'eventually', 'welcome', 'runs', "who's", 'uneven', 'worse', 'source', 'comedic', 'cool', 'sad', 'heavy', 'old-fashioned', 'interested', 'indie', 'long,', 'cannot', 'meant', 'sexual', 'pieces', 'battle', "she's", 'leading', 'ability', 'fight', 'tedious', 'step', 'Big', 'lots', 'sheer', 'four', 'tells', 'opening', 'Marvel', 'entertaining,', 'World', 'change', 'amazing', 'sharp', 'Peter', 'Perhaps', 'acted', 'day', 'moves', 'Good', 'terms', 'writer-director', 'chance', 'original.', 'original,', 'turned', 'empty', 'Most', 'view', 'road', 'provide', 'awful', 'humour', 'there.', 'pleasure', 'thrilling', 'including', 'touching', 'comedies', 'leaving', 'on.', 'male', 'ensemble', 'studio', 'increasingly', 'late', 'in.', 'standard', 'Lee', 'name', 'formulaic', 'sounds', 'Who', 'touch', 'spectacle', 'sports', 'thinking', 'dead', 'return', 'wanted', "What's", 'extraordinary', 'subtle', 'entertainment.', 'CGI', 'thin', 'near', 'serves', 'supporting', 'number', 'missing', 'mood', 'happy', 'gripping', 'tribute', 'understand', 'adults', 'coming-of-age', 'money', 'Every', 'Me', 'poor', 'finding', 'middle', 'white', 'totally', 'straight', 'space', 'seriously', 'begins', 'minutes.', 'twist', 'warm', 'formula', 'either.', 'tired', 'Maybe', 'best,', 'insight', 'humor,', 'performance,', 'average', 'puts', 'designed', 'cut', 'Content', 'blockbuster', 'waste', 'figure', 'longer', 'voice', 'place.', 'successful', 'say,', 'intense', 'Yet', "director's", 'Another', 'opportunity', '3D', 'ridiculous', 'doubt', 'material.', 'honest', 'witty', 'future', 'Dark', 'mediocre', 'truth', 'entertaining.', 'moments.', "one's", 'forced', 'effect', 'rock', 'character,', 'frequently', 'presence', 'showing', 'consistently', 'tough', 'adult', 'events', 'appealing', 'artistic', 'him.', 'imagine', 'Life', 'throughout', 'possibly', 'pleasant', 'trip', 'pull', 'stay', 'particular', 'theater', 'creates', 'Jack', 'directorial', 'for.', 'forgettable', 'alone', "hasn't", 'unexpected', 'talking', 'stop', 'plot,', 'odd', 'Allen', 'technical', 'thoughtful', 'holds', 'details', 'team', 'These', 'Scott', 'stupid', 'before.', 'living', 'turning', 'seemingly', 'cultural', 'appreciate', 'made.', 'screen,', 'thrills', 'intimate', 'creepy', 'dialogue,', 'Has', 'days', 'force', 'poignant', 'superb', 'physical', 'involving', 'essentially', 'entry', 'off.', 'script.', 'clichï¿½ï¿½s', 'out,', 'fairy', 'equivalent', 'balance', 'room', 'brought', 'air', 'footage', 'science', 'others', 'setting', 'basic', 'spend', 'reminder', 'together.', 'passion', 'love,', 'convincing', 'combination', 'imagination', 'Bond', 'gorgeous', 'love.', 'bloody', 'atmosphere', 'Woody', 'project', 'episode', "haven't", 'read', 'five', 'poorly', 'mess.', 'offering', 'spectacular', 'grand', 'George', 'adds', 'fiction', 'Everything', 'not.', 'creating', 'Once', 'value', 'character.', 'painfully', 'soon', 'Nothing', 'weight', 'disturbing', 'Kevin', 'recommend', 'contrived', 'enough,', 'incredibly', 'world,', 'joy', 'Jason', 'body', 'towards', 'involved', 'forget', 'Watching', 'Adam', 'edge', 'willing', 'lets', 'stands', 'usually', 'deal', 'score', 'person', 'continues', 'unlikely', 'came', 'but,', 'weak', 'humor.', 'King', 'appears', 'plot.', 'tale.', 'vehicle', 'shame', 'efforts', 'heartfelt', 'year,', 'Never', 'success', "man's", 'holiday', 'Bad', 'popcorn', 'hand', 'Great', 'move', 'on,', 'class', 'blood', 'Yes,', 'equal', 'soul', 'talk', 'stage', 'unfunny', 'Only', 'missed', 'Parents', 'prove', 'due', 'emotion', 'again,', 'bad.', 'lazy', 'humanity', 'product', 'sentimental', 'Movie', 'Why', 'lovely', 'waiting', 'laughs,', 'problems', 'beneath', 'Part', 'enjoyed', 'further', 'own.', 'thing.', 'allows', '90', 'stylish', 'car', 'quiet', 'cinematography', 'remember', 'best.', 'emotions', 'Little', 'went', 'them,', 'cute', 'deserve', 'sight', 'mildly', 'screenwriter', 'apart', 'era', 'frustrating', 'girl', 'scenes,', 'minor', 'says', 'shallow', 'type', 'current', 'laughs.', 'natural', 'quirky', 'fast', 'portrayal', 'promise', 'cult', 'smart,', 'wildly', 'striking', 'words', 'impact', 'awkward', 'fantastic', 'delightful', 'provocative', 'structure', 'exploration', 'dance', 'writer', 'blend', 'design', 'Much', 'inventive', 'De', 'depiction', 'endless', 'break', 'present', 'revenge', 'struggles', 'action.', 'hell', 'bad,', 'promising', 'meets', 'group', 'wonderfully', 'suggests', 'word', 'using', 'master', 'oddly', 'director,', 'Steve', 'flawed', 'joke', 'occasional', 'Where', 'earnest', 'lies', 'monster', 'Ben', 'theme', 'riveting', 'times.', 'winning', 'la', 'us.', 'much.', 'disaster', 'up,', 'equally', 'engrossing', 'pacing', 'go,', 'about.', 'popular', 'watchable', 'Your', 'worked', 'affecting', 'complicated', 'refreshing', 'results', 'House', 'becoming', 'key', 'saying', 'credits', 'open', 'this,', 'teenage', 'heart,', 'York', 'pointless', 'begin', 'tragedy', 'dream', 'stuck', 'please', 'deeper', 'shock', 'typical', 'America', 'strength', 'order', '3', 'content', 'viewing', 'else.', 'fear', 'right.', 'mainstream', 'taste', 'ground', 'tragic', 'to.', 'child', 'Ultimately,', 'reach', 'history.', 'pay', 'Smith', 'weird', 'likable', 'carry', 'hoping', 'broad', 'actress', 'struggle', 'outside', 'shots', 'hands', 'bigger', 'artist', 'politics', 'surface', 'basically', 'common', 'keeping', 'franchise.', 'capture', 'delivering', 'grim', 'brutal', 'directors', 'Girl', 'Best', 'over.', "He's", 'credit', 'masterpiece', 'guys', 'haunting', 'derivative', 'featuring', 'effects,', 'real-life', 'motion', 'loud', 'point.', 'Steven', 'effectively', 'slapstick', 'expectations', 'style,', 'commentary', 'free', 'thrill', 'miss', 'do.', 'detail', 'somewhere', 'desperate', 'desperately', 'His', 'remain', 'essential', 'hear', 'night', 'slick', 'core', 'spent', 'casting', 'hate', 'known', 'loved', 'raw', 'magical', 'you,', 'picture.', 'rise', 'Martin', '(Full', 'meaning', 'addition', 'material,', 'contains', 'superior', 'audience,', 'huge', 'Moore', 'quietly', 'built', 'similar', 'Without', 'White', 'generally', 'wild', 'delivered', 'mixed', 'ones', 'total', 'various', 'City', 'course', 'follows', 'itself,', 'surely', "You'll", 'reveals', 'builds', 'delight', 'follow', 'there,', 'display', 'house', 'brain', 'expected', 'spy', 'Richard', 'took', 'Those', 'wait', 'B-movie', 'state', 'bizarre', 'share', 'realize', "shouldn't", 'box', 'necessary', 'Dead', 'lesson', 'people.', 'nowhere', 'in,', 'intelligence', 'profound', 'brilliantly', 'supernatural', 'violence,', 'badly', 'whatever', 'constantly', 'Both', 'win', 'dull,', 'big,', 'acting,', 'ride.', 'tense', 'parents', 'affair', 'capable', 'tale,', 'celebration', 'presents', 'scares', '10', 'downright', 'desire', 'account', 'incredible', 'me,', 'watch,', 'fact,', 'homage', 'zombie', 'beginning', 'over-the-top', 'wears', 'editing', 'faith', 'cold', 'target', 'flick.', 'genre,', 'shocking', 'business', 'punch', 'roles', 'bold', 'inner', 'me.', 'not,', 'imaginative', 'manage', 'saga', '...the', 'showcase', 'country', 'with.', 'Wars', 'classic.', 'actors,', 'years,', 'slice', 'carries', 'bunch', 'hot', 'traditional', 'climax', 'examination', 'news', 'bits', 'Sandler', 'filmmaking.', 'yes,', 'boy', 'execution', 'Christopher', 'finest', 'minute', 'ugly', 'search', 'feel-good', 'Here', 'build', 'vivid', 'Harry', 'series.', 'general', 'goofy', "they've", 'match', 'dialogue.', 'cartoon', "weren't", 'absurd', 'Instead', 'accomplished', 'Two', 'style.', 'painful', "children's", 'Now', 'winds', 'commercial', 'beautiful,', 'convoluted', 'respect', 'slight', 'unusual', 'dazzling', 'bright', 'e', 'liked', 'Mark', 'documentary.', 'lame', 'treat', 'chase', 'large', 'endearing', 'relationships', 'wasted', 'understanding', 'evil', 'Chris', 'aims', 'soap', 'experience,', 'uninspired', 'before,', '20', 'obviously', 'slowly', 'starring', 'Spielberg', 'pulls', 'undeniably', 'sitting', "story's", 'o', 'triumph', 'collection', 'learn', 'kid', 'modest', '(or', 'Red', 'substance', 'soundtrack', 'sequence', 'mere', 'drawn', 'noir', 'intended', 'brand', 'strangely', 'sequel.', 'Pixar', 'walk', 'happen', 'Mike', 'fit', 'seemed', 'cinema,', 'Or', 'digital', 'giant', 'directing', 'logic', 'through.', 'onto', 'forward', 'friends', 'relatively', 'picture,', 'Death', 'cynical', 'excuse', 'terribly', 'melodramatic', 'thinks', 'caught', 'younger', 'great,', 'frame', 'away.', 'necessarily', 'memory', 'ask', 'charismatic', 'made,', 'grace', 'mark', 'heavily', 'remarkably', 'gentle', 'earlier', 'produced', 'am', 'sweet,', 'public', 'us,', 'Still,', 'Do', 'sense.', 'potentially', 'executed', 'Jones', 'funniest', 'front', 'growing', 'cheesy', 'storyline', 'famous', 'television', 'Story', 'premise,', 'kept', 'Tim', 'favorite', 'Van', 'intensity', 'nostalgia', 'conclusion', 'Up', 'songs', 'places', 'authentic', 'bringing', 'repetitive', 'talents', 'pass', 'people,', 'excitement', 'boring.', 'ultimate', 'element', 'reminds', 'perspective', 'long.', 'nasty', 'saw', 'sophisticated', 'failure', 'gritty', 'course,', 'visceral', 'plain', 'stick', 'point,', 'together,', 'part,', 'music,', "characters'", 'aesthetic', 'dry', 'daring', 'absorbing', 'intentions', 'budget', 'absolute', 'dark,', 'ready', 'falling', 'pair', 'unsettling', 'bleak', 'Anderson', 'greater', 'wholly', 'ending.', 'Green', 'predictable,', 'considerable', 'skill', 'development', 'guilty', 'About', 'gay', 'safe', 'inevitable', 'fashion', 'seen.', 'center', 'annoying', 'meditation', 'older', 'grows', 'pretentious', 'man.', 'art.', 'finale', 'unnecessary', 'Time', 'silly,', 'lose', 'boasts', 'Jim', "I'll", 'She', 'captivating', 'messy', 'laughing', 'romp', 'superficial', 'slasher', 'mind.', 'trouble', 'misses', 'pleasures', 'justice', 'failed', 'pain', 'serve', 'down.', 'day.', 'devoid', 'English', 'dull.', 'plotting', 'consider', 'Ryan', 'muddled', 'treatment', '3-D', 'Daniel', 'narrative.', 'explores', 'wondering', 'place,', 'closer', 'building', 'imagery', 'dog', 'Western', 'rom-com', 'thought-provoking', 'mature', 'cliches', 'aimed', 'challenging', 'realism', 'interesting.', 'fighting', 'unlike', 'race', 'knowing', 'process', 'personality', 'else,', 'avoid', 'language', 'Bay', 'Stephen', 'First', 'relies', 'Fans', 'effects.', 'inspirational', 'Jennifer', 'Instead,', 'depressing', 'Batman', 'other.', 'violence.', 'committed', '[The', 'justify', 'virtually', 'buy', 'II', 'unfortunately', 'expecting', 'girls', 'crazy', 'storytelling.', 'answer', 'packed', 'fail', 'doc', 'However,', 'nuanced', 'direction,', 'jokes,', 'identity', 'fans.', 'insightful', 'Days', 'realistic', 'so.', 'installment', 'sensitive', 'alive', 'killer', 'ordinary', 'confused', 'herself', 'albeit', 'food', 'presented', 'y', 'extreme', 'career.', 'tropes', 'individual', 'title.', '30', 'level.', 'marks', 'multiple', 'appear', 'putting', 'myself', 'timely', 'Joe', 'gore', 'heroes', 'Stone', 'achieves', 'Men', 'relentlessly', 'romance,', 'demands', 'surprises', 'crude', "character's", 'entertainment,', 'stuff.', 'filmed', 'society', 'funnier', 'go.', 'drug', 'kick', 'etc.', 'protagonist', 'former', 'whom', 'ambition', 'universe', 'Potter', 'Bill', 'conflict', 'stuff,', 'Any', 'now.', 'company', 'clunky', 'Sam', 'flaws', 'loss', 'note', 'Mary', 'overlong', 'religious', 'colorful', 'song', 'clumsy', 'fair', 'literally', 'potent', 'believable', 'heist', 'sum', 'overcome', 'admirable', 'nicely', 'soul.', 'party', 'admire', 'sadly', 'hollow', 'Brothers', 'touches', 'brothers', 'helps', 'Get', 'so,', 'tour', 'suggest', 'are,', 'relentless', 'date', 'straightforward', 'fill', 'issue', 'root', 'charm.', 'coherent', 'suspenseful', "Allen's", 'lively', 'mainly', 'released', 'loose', 'works.', 'intellectual', 'uncomfortable', 'charisma', 'genius', 'meaningful', 'terrifying', 'handful', 'writers', 'documentary,', 'low-budget', 'Whatever', 'control', 'managed', 'limited', 'likes', 'constant', 'throughout.', 'situations', 'caper', 'achievement', 'situation', 'sustain', 'focused', 'served', "Hollywood's", 'morality', 'decide', 'spin', 'reaches', 'confusing', 'exhilarating', 'sympathetic', 'wanting', 'fans,', 'nonetheless', 'spite', 'values', 'interesting,', 'marriage', "We're", 'service', 'tears', 'corporate', 'title,', 'concerned', 'inspiring', 'Go', 'hours.', 'hugely', 'decidedly', 'scenes.', 'dramatically', 'cop', 'preposterous', 'boys', 'vampire', 'Jackson', 'considering', 'Johnson', 'city', 'now,', 'draw', 'computer', 'elegant', 'beloved', 'tender', 'twisted', 'routine', 'busy', 'speak', 'kids.', 'Be', 'simultaneously', 'melancholy', 'see.', 'satisfy', 'spiritual', 'parody', 'originality', 'low-key', "America's", 'losing', "Let's", 'murder', 'dynamic', 'Feels', 'release', 'references', 'false', 'direction.', 'writer/director', 'price', 'much,', 'Writer-director', 'allow', 'engage', 'extended', 'Out', 'does.', 'God', 'history,', 'outstanding', 'knew', 'significant', 'subject.', 'Can', 'likeable', "Here's", 'ludicrous', 'Coen', 'buddy', 'fable', 'Eastwood', 'unpleasant', 'reveal', 'chilling', 'heard', 'elevate', 'heavy-handed', 'thrown', 'gave', 'Earth', 'Sadly,', 'premise.', 'role.', 'delicate', 'Cruise', 'fare', 'graphic', 'poetic', 'land', 'escape', 'matter.', 'Bourne', 'thing,', 'craft', 'date.', 'series,', 'paced', 'charming,', 'catch', 'complexity', 'simplistic', 'far.', 'jump', 'family.', 'rendered', 'La', 'manner', 'her.', 'show.', 'pictures', 'mention', 'system', 'century', 'police', 'friendship', 'seriously.', 'satirical', 'Lost', 'say.', 'melodrama.', 'man,', 'sincere', 'disappointment.', 'indeed', 'alien', 'Robin', 'packs', 'act,', 'throw', 'Japanese', 'unable', 'energetic', 'theatrical', 'predecessor,', 'carefully', 'exploitation', 'case,', 'explore', 'achieve', 'grow', 'underwhelming', 'Sometimes', 'generation', 'Well,', 'smile', 'urban', 'tone,', 'Ferrell', 'acted,', 'choice', 'rousing', 'outrageous', 'hours,', 'bittersweet', 'Whether', 'connect', "Disney's", 'relevant', 'held', 'stays', 'determined', 'Because', 'following', "world's", 'Russell', 'notion', 'color', 'right,', 'ways,', 'stale', 'beats', 'aspect', 'impression', 'damn', 'sloppy', 'Wild', 'drive', 'members', 'amounts', 'sitcom', 'bored', 'embrace', 'Given', 'shot,', 'harrowing', 'terror', 'practically', 'worthwhile', 'plus', 'Old', 'ways.', 'farce', 'adventure.', 'assured', 'Book', 'faces', 'providing', 'scene.', 'throws', 'precisely', 'ever.', 'engaging,', 'been.', 'considered', 'mixture', 'act.', 'conventions', 'bloated', 'adolescent', 'Twilight', 'fits', 'revealing', 'narrative,', 'vs.', 'faithful', 'disappointment', 'ghost', 'reasons', 'opera', 'sequences,', 'around.', 'romance.', 'asks', 'pays', 'superbly', 'proof', 'Cage', 'West', 'exploring', 'devastating', 'live-action', 'territory', 'well-acted', 'aspects', 'astonishing', 'unexpectedly', 'role,', 'lovers', 'Her', 'nobody', 'letting', 'mental', 'cover', 'Three', 'Jane', 'requires', 'bar', 'works,', 'throwback', 'flat.', 'Overall,', 'mild', 'explain', 'jokes.', 'Very', 'misguided', 'understated', 'remotely', 'would-be', 'See', 'are.', 'feelings', 'exceptional', 'memory.', 'rough', 'disappointingly', 'gross-out', 'moving,', 'come.', 'cautionary', 'positive', 'stirring', 'emerges', 'other,', 'Ms.', 'Neither', 'sense,', 'kill', 'Boy', 'decades', 'to,', 'wit,', 'demonstrates', 'million', 'test', 'family,', 'suffering', 'Howard', 'includes', 'predecessor.', 'resulting', 'worse.', 'literary', 'boring,', 'utter', 'del', 'spark', 'McCarthy', 'wrong.', 'then,', 'art,', 'reasonably', 'apparently', 'Available)', 'Are', 'level,', 'context', 'ideas,', 'franchise,', 'game.', 'back.', 'Before', 'today.', 'spends', 'ought', 'combines', 'write', 'rises', 'actors.', 'media', 'Burton', 'image', 'loving', 'acts', 'range', 'constructed', 'scene,', 'expertly', 'Thomas', 'purpose', 'wrapped', 'Ghost', 'Other', 'returns', 'realized', 'subversive', 'aside,', 'unfortunate', 'richly', 'Far', 'warmth', 'double', 'band', 'big-screen', 'Williams', 'curious', 'for,', 'choices', 'available)', 'masterful', 'vital', 'do,', 'afraid', 'gifted', 'him,', 'show,', 'follow-up', 'Lawrence', 'book,', 'heads', 'sequels', 'avoids', 'forgotten', 'shines', 'competent', 'Iron', 'playful', 'Full', 'father', 'feminist', 'raises', 'filmmaking,', 'meandering', 'ï¿½ï¿½', 'Think', "today's", 'draws', 'behavior', 'path', 'brief', 'frightening', "Anderson's", 'ingredients', 'gift', 'specific', 'Captain', 'confident', 'Frank', 'stops', 'off,', 'Miss', 'better,', 'technology', 'subject,', 'ode', 'sees', 'lackluster', 'watched', 'red', 'inspire', 'Almost', 'scale', 'Wilson', 'forgettable.', 'substance.', 'trapped', 'strictly', 'metaphor', 'entertain', 'grief', 'handsome', 'too,', 'critical', 'Then', 'sexy', 'heartbreaking', 'convey', 'extra', 'tiresome', 'brave', 'Unlike', 'driven', 'beat', 'sentimentality', 'benefits', 'Age', 'troubled', 'broken', 'darkly', "Lee's", 'Johnny', 'magnificent', 'thrillers', 'represents', 'challenge', 'moviegoers', 'capturing', 'Game', 'nevertheless', 'breezy', '...an', 'universal', 'tradition', 'domestic', 'Shyamalan', 'awfully', 'incoherent', 'importance', 'iconic', 'nostalgic', 'holes', 'amid', 'South', 'breathtaking', 'en', 'deadly', 'childhood', 'moment.', 'thematic', 'inherent', 'Hard', 'offensive', 'momentum', 'storytelling,', 'overwrought', 'more,', 'Paris', 'community', 'larger', 'Die', 'cause', 'novel,', 'connection', 'elaborate', 'allowed', 'direct', 'chapter', 'cuts', 'survival', 'fourth', 'Street', 'subtlety', 'crew', 'talent.', 'degree', 'masterpiece.', 'landscape', 'next.', 'surreal', 'statement', 'Movies', 'struggling', 'manipulative', "You're", 'refreshingly', 'tales', 'push', 'empathy', 'ago.', 'spoof', 'effect.', '"A', 'precious', 'silliness', 'Does', 'professional', 'Christian', 'mindless', 'exciting,', 'comic-book', 'remind', 'ten', 'vibrant', 'skin', 'sell', 'Beyond', 'destined', 'Charlie', 'effort.', 'clichï¿½ï¿½', 'performers', 'loud,', 'distinctive', 'comfort', '(the', 'imagination.', 'roll', 'generate', 'Something', 'glorious', 'first-time', "audience's", 'done.', 'clever,', 'fitting', 'by-the-numbers', 'flicks', 'se', 'conclusion.', 'Guy', 'deftly', 'instead.', 'lives.', 'shooting', 'dangerous', 'levels', 'allowing', 'gory', 'crisis', 'international', 'camp', 'ideas.', 'notes', 'music.', 'inspiration', 'compelling.', 'racial', 'over,', 'sequel,', 'home.', 'halfway', 'six', 'offbeat', 'happened', "'80s", 'possible.', 'great.', 'helped', 'unless', 'cross', 'gangster', 'required', 'profoundly', 'mesmerizing', 'sporadically', 'subjects', 'stellar', 'himself.', 'list', 'memories', 'haunted', 'angry', 'compared', 'dreary', 'strike', 'book.', 'curiously', 'horror.', 'sex,', 'shifts', 'patience', 'ironic', '[It]', 'cry', 'hilarious,', 'power.', 'initially', 'Tarantino', 'scare', 'fascinating,', 'favor', 'play.', 'onscreen', 'Bruce', 'anything,', 'lessons', 'suspense,', 'classic,', 'check', 'sympathy', 'Greek', 'secret', 'aware', 'testament', 'stereotypes', 'clichï¿½ï¿½d', 'glimpse', 'successfully', 'compelling,', 'fire', 'horror,', 'quick', 'enjoyable,', 'train', 'buried', 'another.', 'atmospheric', 'animation,', 'office', 'Anyone', 'Roger', 'Washington', 'impact.', 'existential', 'bound', 'yet,', 'Hitchcock', 'length', 'Violence,', 'refuses', 'finish', 'melodrama,', 'guess', 'impressively', 'letter', 'sticks', 'Less', 'philosophical', 'sure,', 'audiences.', 'amusing,', 'familiar,', 'stakes', 'suspense.', '100', 'hopes', 'commitment', 'Witch', 'Take', 'uplifting', 'blast', 'affection', 'humans', 'earns', 'nonsense', 'information', 'lifeless', 'wide', 'Four', 'nightmare', 'celebrity', 'everyday', 'Spider-Man', 'parts.', 'Land', 'western', 'Halloween', 'sign', 'experiment', 'pedestrian', 'death.', 'Queen', 'Lady', 'Rock', 'mother', 'intelligent,', 'Ford', 'experiences', 'forces', 'attitude', 'speaks', 'continue', 'novel.', 'feature-length', 'exquisite', 'flick,', 'undoubtedly', 'trick', 'casual', 'difference', 'ahead', 'eccentric', 'yet.', 'folks', 'predictable.', 'admit', 'campy', 'tone.', 'ending,', 'passionate', 'theater.', 'Kate', 'pushes', 'bottom', 'crowd', 'Woman', 'documentaries', 'Ron', 'finish.', 'directs', 'changes', 'treats', 'won', 'Road', 'flat,', 'Jon', 'stretches', 'energy,', 'season', 'mess,', 'Nicolas', 'Wes', 'industry', 'lasting', 'two-hour', 'hair', 'Crowe', 'tonal', 'care.', 'around,', 'face.', 'Rather', 'darker', 'dose', 'tension,', 'ideal', 'eyes.', 'Brian', 'well-made', 'conceived', 'intrigue', 'led', 'Everyone', 'villain', 'breaks', 'short,', 'dreams', 'flaws,', 'Australian', 'But,', 'least,', 'initial', 'disjointed', 'chaotic', 'Rob', 'exists', 'Their', 'trilogy', 'dramedy', 'water', 'diverting', 'violent,', 'stylistic', 'People', 'acting.', 'started', 'Matt', 'outing', 'raunchy', 'count', 'proper', 'Streep', 'mayhem', 'figures', 'dread', 'along.', 'Let', 'spot', 'unfolds', 'Matthew', 'actresses', 'Amy', 'execution.', 'books', 'gender', 'attention.', 'charms', 'players', 'surprised', 'evocative', 'Knight', 'Call', 'gradually', 'viewing.', 'argument', 'wrote', 'describe', 'screwball', 'attractive', 'calling', 'numbers', 'cutting', 'unfortunately,', 'random', 'answers', 'status', 'deliberately', 'dirty', 'edited', 'massive', 'rush', 'sea', 'tight', 'breaking', 'startling', 'detailed', 'redemption', 'displays', 'feature,', 'typically', 'charm,', 'with,', 'rewarding', 'message.', 'Blue', 'big-budget', "She's", 'whimsical', 'matter,', 'slow,', 'steals', 'simple,', 'Home', 'quest', 'Film', 'visuals,', 'Park', 'added', 'support', 'Anna', 'military', 'shake', 'exist', 'technically', 'stretched', 'raise', 'trust', 'juvenile', 'of.', "it'll", 'debut,', 'evidence', 'vast', 'insights', 'irony', 'glossy', 'authenticity', 'emotion.', 'transcends', 'deft', 'later', 'old-school', 'Funny', 'closing', 'cleverly', 'William', 'martial', 'Saturday', 'scenario', 'grounded', 'saving', 'comfortable', 'Baby', 'star.', 'credible', 'higher', 'apparent', 'moment,', 'sly', 'legend', 'local', 'elevates', 'reading', 'teens', 'convince', 'noble', 'settles', 'courage', 'pic', 'Young', 'Cold', 'Alex', 'whole.', 'themselves.', 'qualities', "Doesn't", 'global', 'stretch', 'machine', 'legendary', 'Affleck', 'Depp', 'tension.', 'real,', 'arts', 'powerfully', 'Cameron', 'leads.', 'looked', 'harder', 'kids,', 'tremendous', 'dramas', 'tends', 'starting', 'flair', 'event', 'strong,', 'focuses', 'breath', 'dealing', 'hint', 'stumbles', 'solid,', 'powerful,', 'London', 'same.', 'Simon', 'layers', 'things,', 'banal', 'Our', 'loves', 'purely', 'firmly', 'described', 'forgive', 'tepid', 'Nick', 'yarn', 'Clooney', 'strikes', 'frustratingly', 'Spike', 'bore.', 'death,', 'risk', 'wishing', 'however', 'Rocky', 'accept', 'mysterious', 'shown', 'Ridley', 'light,', 'Morgan', 'sensibility', 'matters', 'sentiment', 'Profanity,', 'tried', 'irritating', 'magic.', 'emphasis', 'Thanks', 'earn', 'down,', 'misfire', 'improvement', 'devoted', 'children.', 'Scorsese', 'age.', 'formula.', 'veers', 'thus', 'Sure,', 'sequences.', 'born', 'aside', 'days.', 'decade', 'hang', 'sharply', 'heroine', 'eager', 'promises', 'which,', 'does,', 'corny', 'primarily', 'obsession', 'new,', 'interviews', 'mass', 'laugh-out-loud', 'consistent', 'string', 'Billy', 'Saw', 'gem', 'sides', 'immediately', 'May', 'scope', 'track', 'unconvincing', 'blow', 'um', 'Happy', 'challenges', 'aggressively', 'scenery', 'hybrid', '15', 'at,', 'Alice', 'deadpan', 'assembled', 'High', 'awesome', 'pleasing', 'finale.', 'moved', 'first-rate', 'trite', 'became', 'true.', 'adding', 'Jeff', 'singular', 'Still', 'did.', 'reality.', 'delightfully', 'produce', 'slog', 'unforgettable', '[An]', 'wear', 'creativity', 'back,', 'seat', 'danger', 'seeking', "They're", 'words,', 'unsatisfying', 'adapted', 'really,', '"I', 'youth', 'scary,', 'hackneyed', 'Dr.', 'stars,', '40', 'another,', 'Pretty', 'understands', 'expect.', 'mistake', 'THE', 'realm', 'own,', 'grown', 'Los', 'bag', 'excessive', 'taut', 'far,', 'clichï¿½ï', 'horrors', 'Golden', 'ring', 'laughter', 'small,', 'Tony', 'moving.', 'failing', 'lesser', 'Clint', 'paying', 'Andrew', 'underlying', 'novelty', 'tricks', 'edgy', "Spielberg's", 'success.', 'pushing', 'asking', 'Jackie', 'send', 'attempting', "Burton's", 'legacy', 'darkness', 'adventure,', 'suddenly', 'undeniable', 'enjoyable.', 'Fantastic', 'predecessor', 'fate', 'things.', 'animals', 'energy.', 'whole,', 'lived', 'artificial', 'women,', 'piece,', 'Ice', 'Wonder', 'science-fiction', 'Better', 'head.', 'erotic', 'accessible', 'resembles', 'through,', 'bear', 'indeed.', 'saves', 'bland,', 'climactic', 'twice', 'background', 'repeatedly', 'Which', 'candy', 'first,', 'humorous', 'independent', 'Lord', 'passable', 'past.', 'know,', 'deliciously', 'effort,', 'Runner', 'dozen', 'meet', 'Italian', 'critics', 'anything.', 'disappointed', 'women.', 'theaters', 'director.', 'Apatow', 'harmless', 'intent', 'compensate', 'photography', 'pace,', 'M.', 'spirit.', 'horrible', "Moore's", 'humanity.', 'actor,', 'son', 'palpable', 'expensive', 'national', 'murky', 'pulled', 'families', 'laugh.', 'satire,', 'porn', 'aiming', 'wise', 'stock', 'wastes', 'drag', 'Kill', 'performed', 'absence', 'something.', 'instant', 'Imagine', 'half-baked', 'essence', 'bore', 'innocent', 'sink', '[the', 'existence', 'holding', 'feat', 'steady', 'poetry', 'None', 'decade.', 'record', 'worse,', 'measure', 'season.', 'staged', 'Bob', 'handled', 'scary.', 'revelations', 'nonsensical', 'saved', 'Mel', 'instantly', 'ones.', 'standards', 'pick', 'Ultimately', 'animal', 'written,', 'conversation', 'denying', 'humour,', 'fine,', 'setup', 'timing', 'ever,', 'European', 'pile', 'intermittently', 'arc', 'was,', 'rules', 'passing', 'Perry', 'surprise.', 'while,', 'half,', 'gotten', 'skills', 'thrills.', "ain't", 'captured', 'street', 'remembered', 'odds', 'game,', 'Hunger', 'distinct', 'crucial', 'passes', 'process.', 'animation.', 'hitting', 'drags', 'plodding', 'Soderbergh', 'tiny', 'anywhere', 'screenwriters', 'inept', 'ranks', 'expectations.', 'delight.', 'vaguely', 'Live', 'Am', 'mad', 'heartwarming', 'flimsy', 'filmmaker.', 'monsters', 'veteran', 'evokes', 'Nicholas', 'Oh,', 'like.', 'hearts', 'Davis', 'movement', 'makers', 'reboot', 'named', 'exceptionally', 'costume', 'soft', 'developed', 'suspect', 'paid', 'easier', 'Secret', 'creation', 'overwhelming', 'reflection', 'heights', 'reduced', 'bitter', 'enormous', 'power,', 'origin', 'handle', 'eat', 'urgency', 'complex,', 'laughed', 'pleasure.', 'summer.', 'Blade', 'opportunity.', 'Miller', 'finely', 'said,', 'whenever', 'sad,', 'rhythm', 'allegory', 'Charles', 'antics', 'Many', 'anger', 'real.', 'compassion', 'screenplay,', 'Jessica', 'Danny', 'Back', 'Trek', 'sincerity', 'disappointed.', "cinema's", 'hints', 'distance', 'torture', 'via', 'Beauty', 'pulp', 'form.', 'culture.', 'spectacle,', 'Long', 'games', '21st', 'timeless', 'line.', 'Girls', 'rote', 'name.', 'trailer', 'potential.', 'flashy', 'resonant', 'creature', 'Nolan', 'una', 'Kids', 'Cooper', 'U.S.', 'suited', 'opens', 'fresh,', 'cool,', "filmmaker's", 'innocence', 'con', "Scott's", 'retains', 'alongside', 'Hill', 'belongs', 'Over', 'hand,', 'notable', 'directly', 'calculated', 'of,', 'praise', 'smartly', 'writing,', 'celebrates', 'honesty', 'Planet', 'movie?', 'endlessly', 'decision', 'enduring', 'mission', 'inert', 'surrounding', 'special.', 'stories,', 'financial', 'weirdly', 'soulless', 'engaged', 'Max', 'brilliance', 'pack', 'interest.', "kids'", 'gently', 'arrives', 'politically', 'conveys', 'enjoy.', 'concert', 'flying', 'whimsy', 'gag', 'at.', 'hidden', 'un', 'observed', 'Elizabeth', 'flashes', 'Reynolds', 'it?', 'spirited', 'confidence', 'directed,', 'disposable', 'stars.', 'done,', 'sublime', 'doomed', 'drives', "let's", 'sweetness', 'audacious', 'Taylor', 'walking', 'ride,', 'conceit', 'Sean', 'effortlessly', 'stronger', 'middle-aged', 'brisk', 'must-see', 'future.', 'well-crafted', 'reminiscent', 'harsh', 'artists', 'idea,', 'action-packed', 'while.', 'well-intentioned', 'talent,', 'overblown', 'ultimately,', '2"', 'cartoonish', 'Transformers', 'embraces', 'wit.', 'amiable', 'nothing.', 'dressed', 'overstuffed', 'America.', 'cinematography,', 'dig', 'titular', 'feast', 'baby', 'cynicism', 'ridiculously', 'bother', 'absurdity', 'children,', 'proceedings', 'sadness', 'unintentionally', 'Few', 'horrific', "Pixar's", 'Visually', 'witty,', 'cruel', 'is]', 'streets', 'wearing', "woman's", 'Evil', 'fast-paced', 'design,', 'Brad', 'enjoying', 'lazy,', 'satisfying.', 'drawing', 'approaches', 'defined', 'feature.', 'True', 'Such', 'revelation', 'delicious', 'steps', 'Wall', 'laughable', 'Wedding', 'numerous', 'subtly', 'forth', 'embarrassing', 'prison', 'frantic', 'lightweight', 'pathos', 'rooting', 'immensely', 'past,', "could've", 'messages', 'maintains', 'efficient', 'focusing', 'collective', 'ended', 'elements,', 'maintain', 'film...', 'puzzle', 'others.', 'X-Men', '12', 'short.', 'invested', 'mixes', 'screenplay.', 'cash', 'shoot', 'minds', 'update', 'School', 'dancing', 'squanders', 'Under', 'Gibson', 'intimacy', 'explosive', 'winner', "Eastwood's", 'Aside', 'happening', 'aim', 'crass', 'Henry', 'relief', 'Watch', 'cinematographer', 'claustrophobic', 'Philip', 'piece.', 'awards', 'fare.', 'Stewart', 'vanity', 'involved.', 'very,', 'fake', 'rings', 'observations', 'develop', 'obnoxious', 'die', 'watchable,', 'leads,', 'alone.', 'nuance', 'belief', 'tend', 'Through', 'notice', 'Welcome', 'roles.', 'college', 'smarter', 'gags,', 'Vaughn', 'like,', 'camera,', 'photographed', 'Dan', 'factor', 'fictional', 'thrills,', 'appreciation', 'Prince', 'morally', 'arguably', 'peculiar', 'one-note', 'conviction', 'today', 'himself,', 'serial', 'sumptuous', 'Right', 'Monster', 'Offers', 'chases', 'considerably', 'awful.', 'sick', 'keen', 'silent', 'substantial', 'travel', 'Damon', 'throughout,', 'moderately', 'expense', 'idea.', 'followed', 'stunt', 'claim', 'stunts', 'intensely', 'hopelessly', 'unconventional', 'parable', 'highlight', 'Island', 'Neeson', 'dysfunctional', 'climax.', 'blame', 'blue', 'singing', 'town', 'strongest', 'fights', 'opposite', 'Fire', 'watching.', 'parts,', 'career,', 'ages.', 'combined', 'ancient', 'fantasy.', 'Wright', 'genres', 'fashion.', 'surface,', 'Witherspoon', 'Judd', 'creatures', 'treated', 'intense,', 'Heart', 'freedom', 'blends', 'appropriate', 'economic', 'ago,', 'Sarah', 'R-rated', 'Alan', 'seldom', 'uneven,', 'anchored', 'excess', 'retro', 'hilarious.', 'sorry', 'both.', 'fault', 'Boys', 'mundane', 'Things', 'Gone', 'bogged', 'driving', 'explosions', 'End', 'grab', 'shameless', 'decided', 'sluggish', 'generous', 'fantasy,', 'loaded', 'kicks', 'depths', 'arresting', 'Jonathan', 'vibe', 'calls', 'presenting', 'discover', 'Into', 'limp', 'serviceable', 'tightly', 'affair.', 'vintage', 'mind,', 'boxing', 'chaos', 'formidable', 'polished', 'artful', 'Anthony', 'grasp', 'Edward', 'wry', 'costumes', 'Gary', 'day,', 'innovative', 'Water', 'throwing', 'memorable.', 'curiosity', 'inevitably', 'hip', 'Hoffman', 'exposition', 'Toro', 'include', "Smith's", 'sweeping', 'chick', 'foreign', '8', 'Family', 'perverse', 'gruesome', 'critique', 'heroic', 'Carrey', 'humour.', 'mystery,', 'modern-day', 'silly.', 'exquisitely', 'signature', 'often,', 'Beautiful', 'hero.', 'crowd-pleasing', 'facts', 'Academy', 'zero', 'valuable', 'underdog', 'payoff', 'Kong', 'wacky', 'along,', 'recognize', 'Next', 'thrilling,', 'Sex', 'adventures', 'drop', 'lurid', 'Room', 'everybody', 'Hot', 'Jerry', 'accurate', 'layered', 'distinctly', 'artistry', 'villains', 'seeks', 'laugh,', 'Todd', 'trash', 'criminal', 'bite', 'enthusiasm', "'70s", 'dreadful', 'primary', 'deals', "Jackson's", 'unusually', 'learned', 'Makes', 'ambitious,', 'middling', 'victim', 'ticket', 'Dawn', 'changing', 'Neil', 'DVD', 'filmmaker,', 'serving', 'Lucas', 'Russian', 'Arthur', 'akin', 'lush', 'marketing', 'Craig', 'shine', 'protagonists', 'rights', 'mythology', 'deserved', 'Julia', 'Hanks', 'And,', 'exhausting', 'hero,', 'oddball', 'touching,', 'camera.', 'ball', 'spending', 'intriguing,', 'speed', 'new.', 'ambitions', 'provided', '[director', 'below', 'comparison', 'mark.', 'goal', 'von', 'attack', 'kinds', 'Keaton', 'Emma', 'start,', 'ridiculous,', '1970s', 'Beautifully', 'Based', 'era.', 'Between', 'suffer', 'standing', 'possibilities', 'wonders', 'scattered', 'Did', 'on-screen', 'sour', 'treat.', "You've", 'voices', 'inability', 'delivery', 'shadow', 'partly', 'unfocused', 'form,', 'honestly', 'offered', 'manic', 'roots', 'seeing.', 'who,', 'dated', 'Jolie', 'dumb,', 'welcome.', 'altogether', 'proud', 'irresistible', 'reaching', 'thoughtful,', 'marked', 'bears', 'Parental', 'war,', 'eyes,', 'shamelessly', 'adults.', 'Hood', 'Andy', "summer's", 'tense,', 'pleasantly', 'Rachel', 'Jr.', 'majority', 'concerns', 'Friday', 'trademark', 'seriously,', 'survive', 'frenetic', 'engaging.', 'satire.', 'sharp,', 'truths', 'convincingly', 'slick,', 'Quentin', 'Downey', 'Fast', 'experienced', 'troubling', 'comedies.', 'cohesive', 'marvelous', 'black-and-white', 'Alas,', 'highlights', 'pitch', 'Seth', 'tad', 'proving', 'half.', 'carried', 'names', 'technique', 'sentimentality.', 'sketch', 'gorgeous,', 'core.', 'spectacularly', 'Year', 'courtroom', 'el', 'watchable.', 'pulling', 'ground.', 'willingness', 'Eddie', 'dubious', 'self-indulgent', 'evident', 'maudlin', 'epic,', 'though.', 'clarity', 'private', 'moody', 'chronicle', 'thick', 'classics', 'wind', 'par', 'Mexican', 'killing', 'leaden', 'stiff', 'skillfully', 'succeed', 'Roberts', 'tragedy.', 'mystery.', 'punch.', 'dare', 'play,', 'gun', 'neat', 'Coppola', 'wicked', 'Could', 'rewards', 'defies', 'eerie', 'depth.', 'rescue', 'can.', 'vision.', 'or,', 'escapist', 'star,', 'Denzel', "life's", 'steam', 'settle', 'Each', 'paints', 'detective', 'colors', 'about,', 'handsomely', 'Simply', 'hole', 'journey.', 'resonance', 'parade', '1980s', 'fellow', 'strives', 'gleefully', 'splendid', 'edges', 'simplicity', 'reality,', 'restrained', 'feet', 'ago', 'sprawling', 'borderline', 'hilariously', 'detail,', 'nonsense.', 'ingenious', 'character-driven', 'spectacle.', 'Larry', 'espionage', '[Director', 'computer-generated', "Scorsese's", 'locations', 'territory.', 'Perfect', "We've", 'backdrop', 'por', '50', 'distract', 'dynamics', 'lights', 'supremely', 'selling', 'episodic', 'stupid,', 'Shrek', 'strange,', 'paint', 'hardcore', 'joke.', 'longing', 'inane', 'co-writer', 'Finding', 'culture,', 'obvious,', 'images,', 'combat', 'McConaughey', 'honest,', 'regular', 'line,', 'warm,', 'scares.', 'quiet,', 'shape', 'greatness', 'taste.', 'Melissa', 'convinced', 'replaced', 'Josh', 'strained', 'Murray', 'minimal', 'days,', 'Chan', 'Jordan', 'disappointing.', 'formal', 'unpredictable', 'sake', 'brutally', 'Carell', 'Superman', 'idiotic', 'racism', 'emerge', 'package', 'retelling', 'Mostly', 'blows', 'subtext', 'masterpiece,', 'strains', 'Dogs', 'tedious,', "King's", 'perfect,', '(if', 'resonate', 'vague', 'asked', 'pity', 'alternately', 'literal', 'problem.', 'Stallone', 'reaction', 'actions', 'youthful', 'subplots', 'forgets', 'Del', 'Roman', 'viewer.', 'gold', 'Chinese', 'com', 'producer', 'Niro', 'episodes', 'Rarely', 'wisdom', 'Kingdom', 'beating', 'shallow,', 'lands', 'Statham', 'introduction', 'affair,', 'see,', 'Blair', 'uniquely', 'nature.', 'legal', 'scripted', 'enchanting', 'naturalistic', 'Shakespeare', 'vulgar', 'Spanish', 'debate', 'threatens', 'dystopian', 'German', 'handles', 'infectious', 'two.', 'angst', "Ocean's", 'established', 'flow', 'truth.', 'marvel', 'gorgeously', 'Dragon', 'Blood', 'war.', 'stupid.', 'once.', 'Mad', 'Hardy', "isn't.", 'secrets', 'solely', 'field', 'fierce', 'implausible', 'Sea', 'resembling', 'definition', 'lingers', 'variety', 'limits', 'model', 'reel', 'hoped', 'screaming', 'Middle', 'away,', 'hopeful', 'masquerading', 'viewers.', 'clean', 'abuse', 'Would', 'Hong', 'giddy', 'cloying', 'shift', 'depends', 'wave', 'balances', 'continually', 'United', 'author', 'lyrical', 'flesh', 'Kidman', 'overlong,', 'Gordon', 'relatable', 'others,', 'score.', 'consequences', 'bursts', 'debut.', 'transcend', 'Jurassic', 'determination', 'sinks', 'Liam', 'highest', 'showcases', 'cost', 'anyway.', 'decisions', 'golden', 'devolves', 'sappy', 'imagination,', 'echoes', 'wooden', 'para', "viewer's", 'smug', 'fly', 'stories.', 'countless', 'perceptive', 'leap', 'vividly', 'camerawork', 'narration', 'fancy', 'Fox', 'Herzog', 'Sparks', 'filme', 'Gyllenhaal', 'enigmatic', 'US', 'toy', 'paper', 'riff', 'rushed', 'Legend', 'confusion', 'fascination', 'blunt', 'grown-up', 'rage', 'fills', 'cerebral', 'compassionate', 'insipid', 'fatal', 'Taken', 'conspiracy', 'seek', 'guilt', 'Takes', 'purpose.', 'eight', 'balanced', "Sandler's", 'archival', 'supposedly', 'Franco', 'Summer', 'affectionate', 'hour,', 'score,', 'Super', 'development.', 'victims', 'arthouse', 'cares', 'tricky', 'signs', 'strongly', 'spooky', 'cliched', 'apocalyptic', 'cliche', 'glory', 'window', 'latter', 'problems.', 'nails', 'warrant', 'rich,', 'sustained', 'internal', 'tear', 'results.', 'changed', 'familiar.', 'Plays', 'copy', 'fat', 'Irish', 'hypnotic', 'tragedy,', 'adaptations', 'bold,', 'guaranteed', 'standout', 'surface.', 'Here,', 'witless', 'admittedly', 'opportunities', 'sorely', 'lends', 'sugar', 'risks', 'Hart', 'gripping,', 'familiarity', 'Al', 'adorable', 'impressive,', 'desperation', 'gross', 'require', 'tonally', 'good-natured', 'Owen', 'renders', 'Comes', 'gimmick', 'fewer', 'Anne', 'influence', 'sensitivity', 'combine', 'society.', 'unnerving', 'law', 'teeth', 'rooted', 'topic', 'similarly', 'Way', 'Terminator', 'message,', 'card', 'wealth', 'teenagers', 'Oscar-winning', 'Made', 'Pirates', 'mishmash', 'serious,', 'face,', 'disaster.', 'depicts', 'appeal.', 'minimalist', 'spinning', 'lousy', 'wife', 'stylized', 'ponderous', 'ignore', 'composed', 'job.', 'set-up', 'realities', 'concept,', 'production,', 'wins', 'Truth', 'depth,', 'Justin', 'undone', 'balancing', 'well-meaning', 'myth', 'hand.', 'depicted', 'gags.', 'precise', 'woefully', 'Shot', 'wall', 'structured', 'prefer', 'cheer', 'lift', 'brilliant,', 'lovingly', 'rising', 'adrenaline', 'Americans', 'insult', 'checking', 'uncomfortably', 'Bullock', 'lingering', 'once,', 'conflicts', 'argue', 'Women', 'visionary', 'strengths', 'turn.', 'Writer/director', 'money.', 'entertained', 'A-list', 'Battle', 'Often', 'surprises.', 'trashy', 'prequel', 'know.', 'first.', 'deliberate', 'como', 'irreverent', "You'd", 'persona', 'happily', 'respectful', 'Don', 'aspires', 'Forget', 'value.', 'dedicated', 'Emily', 'toxic', 'absurdist', 'Patrick', 'expression', 'North', 'knowledge', 'brutality', 'Joel', 'fish', 'battles', 'side.', 'No,', 'snappy', 'worn', 'Ray', 'Cars', 'dying', 'fleeting', 'spell', 'raised', 'suburban', 'task', 'look,', 'urgent', 'rural', 'bare', 'outright', 'achingly', 'meat', 'Deadpool', 'Unless', 'result,', 'generates', 'endure', 'perhaps,', 'ridiculous.', 'weakest', 'emotions.', 'meticulously', 'will.', 'macho', 'Foster', 'elevated', 'shared', 'intimate,', 'careful', 'Walk', 'Make', 'sorts', 'closely', 'recalls', 'lower', 'shaky', 'Jake', 'self-conscious', 'trio', 'horse', '80', 'laid', 'somebody', 'presentation', 'roll.', 'existence.', 'thoughts', 'was.', 'horribly', 'Span', 'offer.', 'sinister', 'disbelief', 'seeming', 'positively', 'Children', 'response', 'Stiller', 'art-house', 'radical', 'enjoyably', 'retread', 'indeed,', 'sufficiently', 'Well', 'madness', 'location', 'Had', 'goes,', 'themselves,', 'procedural', 'portrays', 'uneasy', 'scores', 'junk', 'Train', 'night.', 'Shades', 'one-liners', 'mysteries', 'scattershot', 'senses', 'effective.', 'comedies,', 'nonetheless.', 'staging', "here's", 'dogs', 'get.', 'sentimental,', 'friend', 'alike.', 'previously', 'kiddie', 'portraying', 'seen,', 'remaining', 'apart.', 'skilled', 'repeat', 'production.', '---', 'appearance', 'admirably', 'raw,', 'government', 'versus', 'es', 'banter', 'illuminating', 'Southern', 'controversial', "everyone's", 'aging', 'narratively', 'questionable', 'expert', 'ambiguous', 'money,', 'tedious.', 'mediocrity.', 'whiff', 'began', '13', 'is:', 'finished', 'self', 'Strong', 'Aaron', 'old,', 'terms,', 'appreciated', 'passed', 'Thompson', 'joyous', 'sensational', 'involves', 'politics,', 'credibility', 'closest', '(a', 'lavish', 'laughably', 'resist', 'viewers,', 'beauty.', 'soldiers', 'protagonist,', 'setting,', 'personalities', 'visuals.', 'her,', 'problematic', 'baseball', 'persuasive', 'hope.', 'exceedingly', 'flash', 'audiences,', 'matches', 'precision', 'perfunctory', 'nifty', 'going.', 'Fallen', 'African', 'tendency', 'insane', 'brains', 'glass', 'dares', 'Cronenberg', 'ambiguity', 'everyone,', 'gothic', 'base', 'stark', 'believes', 'super', 'Country', 'grotesque', 'spare', 'learning', 'advantage', 'theatre', 'grace.', 'achievement.', 'hands,', 'fashioned', "who've", 'mediocrity', 'Santa', 'mistakes', 'round', 'lives,', 'rolling', '20th', 'facing', 'ages', 'loneliness', 'personality.', 'indictment', "doesn't.", 'fiction.', 'pace.', 'Meryl', 'regardless', 'witness', 'graceful', 'evoke', 'addiction', 'Real', 'forever', 'idiosyncratic', 'guns', 'medium', 'mouth', 'unfunny,', 'have.', 'grisly', 'stunningly', 'unimaginative', 'achieved', 'levels.', '1', 'Since', "original's", 'chuckles', "women's", 'gloriously', 'issues,', 'resolution', 'critic', 'fascinating.', 'muted', 'Furious', 'casts', 'relying', 'Theron', 'assault', 'Being', "we'll", 'think,', 'soundtrack.', 'air.', 'clichï¿½ï¿½s,', 'age,', 'adaptation.', 'Hugh', 'weaves', 'mean-spirited', 'expect,', 'alive.', 'side,', 'familial', 'Give', 'atmosphere,', 'shockingly', "Isn't", 'exciting.', 'sobering', 'humane', 'well-worn', 'prepared', 'sin', 'length.', 'should.', 'substance,', 'soulful', 'guessing', 'layer', 'elements.', 'endeavor', 'awkwardly', 'PG-13', 'stuffed', 'droll', 'enthralling', 'shiny', 'squarely', 'rely', 'Five', 'unflinching', 'Hunter', 'Reese', 'walks', 'Iraq', 'undermines', 'pale', 'Reeves', 'Phoenix', 'solidly', 'relationships.', 'Down', 'Colin', 'charged', "actor's", 'fan,', 'beauty,', 'told,', 'cable', 'hanging', 'crammed', 'so-called', 'glad', 'students', 'Hollywood.', 'invites', 'sudden', 'definitive', 'awful,', 'Malick', 'romcom', 'helping', 'flaws.', 'fears', 'satisfying,', 'viewed', 'Marshall', 'eye-popping', 'knack', 'noise', 'century.', 'clumsily', 'feel,', 'frame.', 'prime', 'Goes', 'seductive', "people's", 'seven', 'Bell', 'racing', 'tones', 'document', 'true,', 'frankly', 'shy', 'start.', 'drab', 'jarring', 'gratuitous', 'hurt', 'center.', 'liberal', "they'll", 'covers', 'producers', 'ingenuity', 'green', 'one-dimensional', 'hide', 'structure,', 'destruction', 'explored', 'depicting', 'Indian', 'Law', 'door', 'formula,', 'cars', 'men,', 'uncompromising', 'Gosling', 'stomach', 'Inside', 'horrifying', 'bond', 'handling', 'flourishes', 'Michelle', 'blockbusters', 'sweetly', 'benefit', 'CG', 'exuberant', 'choose', 'intentions,', 'top-notch', 'tame', 'Party', 'Horror', 'bracing', 'disparate', 'met', 'dense', 'Toy', 'lean', "Shyamalan's", 'gore,', 'comparison.', 'strain', 'last.', 'Broadway', '2,', 'diversion', 'blood,', 'wannabe', 'Snow', 'creepy,', 'returning', 'reputation', 'excited', 'pretend', 'smoothly', 'emotion,', 'Lifetime', 'lot.', 'ourselves', 'enjoyment', 'streak', 'unbelievable', 'B', 'hard,', 'universe.', 'feel.', 'mash-up', 'bodies', 'suitable', 'Trier', 'cold,', 'chosen', 'scares,', 'ongoing', 'hell.', 'weird,', 'enterprise', 'strikingly', 'qualify', 'repeated', 'recall', 'possibility', 'exotic', 'nowhere.', 'problems,', 'Carter', 'messy,', 'pop-culture', 'pairing', 'bombastic', 'ice', 'variation', 'frequent', 'invasion', 'somber', 'ugly,', 'unremarkable', 'worst,', 'angle', 'date,', 'revealed', 'Angeles', 'shot.', 'Quite', 'invention', 'edge.', 'effective,', 'themes,', 'Creed', 'climax,', 'noisy', 'Space']
    #calculatingthe word occurrence
    
    word_occurrences = {}
    for word in vocab_list: #     positive     neg1tive
        word_occurrences[word] = {'fresh': 0, 'rotten': 0}
        for document in train_set:
            if word in document[1].split():
                sentiment = document[0]
                if sentiment=="Freshness":
                    pass
                else:
                    word_occurrences[word][sentiment] += 1
    # lets do the calculations
    # Calculate ClassProbabilities
    class_probabilities = ClassProbabilities(train_set)
    print("Class Probabilities:",class_probabilities) # prior probability.
    
    conditional_probabilities = word_conditional_probability(train_set, vocab_list, word_occurrences)
    print("Conditional Probabilities:")
    for word in conditional_probabilities: print(word,":",conditional_probabilities[word])
    # Evaluate the accuracy of the classifier on the development set
    dev_accuracy = evaluate(dev_set, vocab_list, class_probabilities, conditional_probabilities)
    print("Accuracy:",dev_accuracy)
    
    word_counts = {}
    dataset=reviews_load('rt_reviews.csv')
    for review in dataset:
        words = review['Review'].split()
        for word in words:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    vocab=[]
    for i in range(10):
        vocab.append(sorted_word_counts[i][0])
    print("Top 10 most frequent words:")
    print(vocab)
    

In [ ]:
# run the code
main()
